In this notebook, we will use [TorchIO](http://torchio.rtfd.io/) and its new [`RSNAMICCAI`](https://torchio.readthedocs.io/datasets.html#rsnamiccai) dataset class to load, preprocess and write the challenge dataset.

In [ ]:
# Import torchio via this notebook 
# https://www.kaggle.com/ohbewise/pip-download-torchio

!pip install --quiet --no-index --find-links ../input/pip-download-torchio/ --requirement ../input/pip-download-torchio/requirements.txt

In [ ]:
from pathlib import Path
import torch
import torchio as tio
import matplotlib.pyplot as plt

plt.rcParams["figure.figsize"] = (12, 10)

In [ ]:
# debug: pass
preprocessing_transforms = (
    tio.ToCanonical(),
    tio.Resample(1, image_interpolation='bspline'),
    tio.Resample('T1w', image_interpolation='nearest'),
    tio.CropOrPad((251,251,150)),
)
preprocess = tio.Compose(preprocessing_transforms)

In [ ]:
root_dir = '/kaggle/input/rsna-miccai-brain-tumor-radiogenomic-classification'

test_set = tio.datasets.RSNAMICCAI(root_dir, train=False, transform=preprocess)

In [ ]:
# Generate dataloaders
batch_size = 1

test_loader = torch.utils.data.DataLoader(test_set,
                                          batch_size=batch_size,
                                          shuffle=False)

print(len(test_loader))

# Generate model and import weights

In [ ]:
import sys
sys.path.append('../input/efficientnetpyttorch3d/EfficientNet-PyTorch-3D')

In [ ]:
from efficientnet_pytorch_3d import EfficientNet3D

model = EfficientNet3D.from_name("efficientnet-b0", override_params={'num_classes': 1}, in_channels=1)
model.cuda()

device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'

In [ ]:
# Load model weights
PATH = '../input/rsna-model-baseline/model_baseline.pt'

model.load_state_dict(torch.load(PATH))
model.eval()

# Inference

In [ ]:
from tqdm import tqdm

sigmoid = torch.nn.Sigmoid()
ls_pred = []

with torch.no_grad():
    for batch in tqdm(test_loader):
    
        # Get input array
        input_flair = batch['FLAIR']['data'].float().to(device)
        #input_t1w = batch['T1w']['data'].float().to(device)
        #input_t1wce = batch['T1wCE']['data'].float().to(device)
        #input_t2w = batch['T2w']['data'].float().to(device)
        
        # Concat input together
        #inputs = torch.cat([input_flair, input_t1w, input_t1wce, input_t2w], dim=0)

        # Feed to the model and get logit
        logit = model(input_flair)
        pred = sigmoid(logit).cpu().detach().numpy().item()
        ls_pred.append(pred)

In [ ]:
# Load sample submission
csv_path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv'

import pandas as pd
submission = pd.read_csv(csv_path)
submission['MGMT_value'] = ls_pred

# Export submission
submission.to_csv('submission.csv',index=False)